In [1]:
import keras
from keras.models import *
from keras.layers import *
from types import MethodType

import os
import cv2
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from os.path import exists, join, basename

IMAGE_ORDERING = 'channels_last'

Using TensorFlow backend.


In [0]:
pretrained_url = "https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [0]:
def get_segmentation_model( input , output ):

	img_input = input
	o = output

	o_shape = Model(img_input , o ).output_shape
	i_shape = Model(img_input , o ).input_shape

	output_height = o_shape[1]
	output_width = o_shape[2]
	input_height = i_shape[1]
	input_width = i_shape[2]
	n_classes = o_shape[3]
	o = (Reshape((   output_height*output_width , -1    )))(o)

	o = (Activation('softmax'))(o)
	model = Model( img_input , o )
	model.output_width = output_width
	model.output_height = output_height
	model.n_classes = n_classes
	model.input_height = input_height
	model.input_width = input_width
	model.model_name = ""

	model.train = MethodType( train , model )
	model.predict_segmentation = MethodType( predict , model )
	model.predict_multiple = MethodType( predict_multiple , model )
	model.evaluate_segmentation = MethodType( evaluate , model )


	return model 

In [0]:
def get_vgg_encoder(input_height=224,input_width=224,pretrained='imagenet'):
    print(type(input_height))
    img_input = Input(shape=(input_height,input_width,3))

    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', data_format=IMAGE_ORDERING )(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2', data_format=IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool', data_format=IMAGE_ORDERING )(x)
    f1 = x
	# Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2', data_format=IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool', data_format=IMAGE_ORDERING )(x)
    f2 = x

	# Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3', data_format=IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool', data_format=IMAGE_ORDERING )(x)
    f3 = x

	# Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3', data_format=IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool', data_format=IMAGE_ORDERING )(x)
    f4 = x

	# Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3', data_format=IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool', data_format=IMAGE_ORDERING )(x)
    f5 = x

	
    if(pretrained == 'imagenet'):
        VGG_Weights_path = keras.utils.get_file(pretrained_url.split("/")[-1],pretrained_url)
        Model(img_input,x).load_weights(VGG_Weights_path)


    return img_input , [f1 , f2 , f3 , f4 , f5 ]


In [0]:
def _unet( n_classes , encoder , l1_skip_conn=True,  input_height=416, input_width=608  ):

	img_input , levels = encoder( input_height=input_height ,  input_width=input_width )
	[f1 , f2 , f3 , f4 , f5 ] = levels 

	o = f4

	o = ( ZeroPadding2D( (1,1) , data_format=IMAGE_ORDERING ))(o)
	o = ( Conv2D(512, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
	o = ( BatchNormalization())(o)

	o = ( UpSampling2D( (2,2), data_format=IMAGE_ORDERING))(o)
	o = ( concatenate([ o ,f3],axis=MERGE_AXIS )  )
	o = ( ZeroPadding2D( (1,1), data_format=IMAGE_ORDERING))(o)
	o = ( Conv2D( 256, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
	o = ( BatchNormalization())(o)

	o = ( UpSampling2D( (2,2), data_format=IMAGE_ORDERING))(o)
	o = ( concatenate([o,f2],axis=MERGE_AXIS ) )
	o = ( ZeroPadding2D((1,1) , data_format=IMAGE_ORDERING ))(o)
	o = ( Conv2D( 128 , (3, 3), padding='valid' , data_format=IMAGE_ORDERING ) )(o)
	o = ( BatchNormalization())(o)

	o = ( UpSampling2D( (2,2), data_format=IMAGE_ORDERING))(o)
	
	if l1_skip_conn:
		o = ( concatenate([o,f1],axis=MERGE_AXIS ) )

	o = ( ZeroPadding2D((1,1)  , data_format=IMAGE_ORDERING ))(o)
	o = ( Conv2D( 64 , (3, 3), padding='valid'  , data_format=IMAGE_ORDERING ))(o)
	o = ( BatchNormalization())(o)

	o =  Conv2D( n_classes , (3, 3) , padding='same', data_format=IMAGE_ORDERING )( o )
	
	model = get_segmentation_model(img_input , o )


	return model


def vgg_unet( n_classes,input_height=416,input_width=608,encoder_level=3):

	model =  _unet( n_classes, get_vgg_encoder,input_height=input_height, input_width=input_width  )
	model.model_name = "vgg_unet"
	return model

In [0]:
cwd = os.getcwd()

In [0]:
dataset_name = join(cwd, 'warwick_qu_dataset_released_2016_07_08')
if not exists(dataset_name + '.zip'):
  get_ipython().system('wget https://warwick.ac.uk/fac/sci/dcs/research/tia/glascontest/download/warwick_qu_dataset_released_2016_07_08.zip')

In [0]:
dataset_dir = join(cwd, 'Warwick QU Dataset (Released 2016_07_08)')
if not exists(dataset_dir):
  get_ipython().system('unzip ' + dataset_name + '.zip')

In [0]:
data = pd.read_csv(join(dataset_dir, 'Grade.csv')) 
data['seg_name'] = [name + '_anno.bmp' for name in data['name']]
data = data[['name', 'seg_name']]
data['name'] = [name + '.bmp' for name in data['name']]
#print(data.columns[2])
data.head()

,name,seg_name
0,testA_1.bmp,testA_1_anno.bmp
1,testA_10.bmp,testA_10_anno.bmp
2,testA_11.bmp,testA_11_anno.bmp
3,testA_12.bmp,testA_12_anno.bmp
4,testA_13.bmp,testA_13_anno.bmp


In [0]:
train_df = data[data.name.str.startswith('train')]
print(train_df.shape)
train_df.head()

(85, 2)


,name,seg_name
80,train_1.bmp,train_1_anno.bmp
81,train_10.bmp,train_10_anno.bmp
82,train_11.bmp,train_11_anno.bmp
83,train_12.bmp,train_12_anno.bmp
84,train_13.bmp,train_13_anno.bmp


In [0]:
valid_df = data[data.name.str.startswith('testA')]
print(valid_df.shape)
valid_df.head()

(60, 2)


,name,seg_name
0,testA_1.bmp,testA_1_anno.bmp
1,testA_10.bmp,testA_10_anno.bmp
2,testA_11.bmp,testA_11_anno.bmp
3,testA_12.bmp,testA_12_anno.bmp
4,testA_13.bmp,testA_13_anno.bmp


In [0]:
for i, j in data.iterrows(): 
    print(j[1]) 

testA_1_anno.bmp
testA_10_anno.bmp
testA_11_anno.bmp
testA_12_anno.bmp
testA_13_anno.bmp
testA_14_anno.bmp
testA_15_anno.bmp
testA_16_anno.bmp
testA_17_anno.bmp
testA_18_anno.bmp
testA_19_anno.bmp
testA_2_anno.bmp
testA_20_anno.bmp
testA_21_anno.bmp
testA_22_anno.bmp
testA_23_anno.bmp
testA_24_anno.bmp
testA_25_anno.bmp
testA_26_anno.bmp
testA_27_anno.bmp
testA_28_anno.bmp
testA_29_anno.bmp
testA_3_anno.bmp
testA_30_anno.bmp
testA_31_anno.bmp
testA_32_anno.bmp
testA_33_anno.bmp
testA_34_anno.bmp
testA_35_anno.bmp
testA_36_anno.bmp
testA_37_anno.bmp
testA_38_anno.bmp
testA_39_anno.bmp
testA_4_anno.bmp
testA_40_anno.bmp
testA_41_anno.bmp
testA_42_anno.bmp
testA_43_anno.bmp
testA_44_anno.bmp
testA_45_anno.bmp
testA_46_anno.bmp
testA_47_anno.bmp
testA_48_anno.bmp
testA_49_anno.bmp
testA_5_anno.bmp
testA_50_anno.bmp
testA_51_anno.bmp
testA_52_anno.bmp
testA_53_anno.bmp
testA_54_anno.bmp
testA_55_anno.bmp
testA_56_anno.bmp
testA_57_anno.bmp
testA_58_anno.bmp
testA_59_anno.bmp
testA_6_anno.bm

In [0]:
import numpy as np
import cv2
import glob
import itertools
import os
from tqdm import tqdm

import random

random.seed(0)
class_colors = [  ( random.randint(0,255),random.randint(0,255),random.randint(0,255)   ) for _ in range(5000)  ]


def get_pairs_from_paths( df):
	ret = []

	for _, names in df.iterrows(names):
		im = os.path.join(dataset_dir, names[0])
		seg = os.path.join( dataset_dir, names[1])
		ret.append((im , seg))

	return ret




def get_image_arr( path , width , height , imgNorm="sub_mean" , odering='channels_first' ):


	if type( path ) is np.ndarray:
		img = path
	else:
		img = cv2.imread(path, 1)

	if imgNorm == "sub_and_divide":
		img = np.float32(cv2.resize(img, ( width , height ))) / 127.5 - 1
	elif imgNorm == "sub_mean":
		img = cv2.resize(img, ( width , height ))
		img = img.astype(np.float32)
		img[:,:,0] -= 103.939
		img[:,:,1] -= 116.779
		img[:,:,2] -= 123.68
		img = img[ : , : , ::-1 ]
	elif imgNorm == "divide":
		img = cv2.resize(img, ( width , height ))
		img = img.astype(np.float32)
		img = img/255.0

	return img






def get_segmentation_arr( path , nClasses ,  width , height , no_reshape=False ):

	seg_labels = np.zeros((  height , width  , nClasses ))
		
	if type( path ) is np.ndarray:
		img = path
	else:
		img = cv2.imread(path, 1)

	img = cv2.resize(img, ( width , height ) , interpolation=cv2.INTER_NEAREST )
	img = img[:, : , 0]

	for c in range(nClasses):
		seg_labels[: , : , c ] = (img == c ).astype(int)


	
	if no_reshape:
		return seg_labels

	seg_labels = np.reshape(seg_labels, ( width*height , nClasses ))
	return seg_labels

def image_segmentation_generator( images_path , segs_path ,  batch_size,  n_classes , input_height , input_width , output_height , output_width):
	
	img_seg_pairs = get_pairs_from_paths( images_path , segs_path )
	random.shuffle( img_seg_pairs )
	zipped = itertools.cycle( img_seg_pairs  )

	while True:
		X = []
		Y = []
		for _ in range( batch_size) :
			im , seg = next(zipped) 

			im = cv2.imread(im , 1 )
			seg = cv2.imread(seg , 1 )

			X.append( get_image_arr(im , input_width , input_height ,odering=IMAGE_ORDERING )  )
			Y.append( get_segmentation_arr( seg , n_classes , output_width , output_height )  )

		yield np.array(X) , np.array(Y)

In [0]:
input_height=224 , 
input_width=224 , 
n_classes=2,
checkpoints_path= '/content', 
epochs = 5,
batch_size = 2,
steps_per_epoch=12,
#optimizer_name='sgd' 


model = vgg_unet(n_classes,input_height,input_width)
   
n_classes = model.n_classes
input_height = model.input_height
input_width = model.input_width
output_height = model.output_height
output_width = model.output_width
model.compile(keras.optimizers.Adam(lr = 0.0001), loss = 'binary_crossentropy', metrics = ['accuracy'])
train_gen = image_segmentation_generator(train_images,train_annotations,batch_size,n_classes,input_height , input_width , output_height,output_width)

val_gen  = image_segmentation_generator( val_images , val_annotations ,  val_batch_size,  n_classes , input_height , input_width , output_height,output_width)

model.fit_generator( train_gen , steps_per_epoch  , validation_split=val_gen , validation_steps=200 ,  epochs=epochs )
if not checkpoints_path is None:
	model.save_weights(checkpoints_path + "." + str(epochs))
	print("saved ",checkpoints_path + ".model." + str(epochs))

<class 'tuple'>


TypeError: ignored